# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core.environment import Environment

import pkg_resources
root_packages=['sklearn']

for m in pkg_resources.working_set:
    if m.project_name.lower() in root_packages:
        print(f"{m.project_name}=={m.version}")
import sys
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install pandas

sklearn==0.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'Capstone-Project-exp'

experiment=Experiment(ws, experiment_name)

project_env=Environment.from_conda_specification(name='project_env',file_path='./conda_dependencies.yml')

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-159184
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-159184


In [3]:

from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
compute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, compute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive import HyperDriveRun
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
import os
import pandas as pd
import sklearn

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2,slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'--C':uniform(0,1),'--max_iter':choice(range(1,1000))})
                                         
if "training" not in os.listdir():
    os.mkdir("./training")

# Running ScriptRun instead of estimator because estimator is depracated
src = ScriptRunConfig(source_directory='.',script='train.py',compute_target=compute_cluster_name,environment=project_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src,hyperparameter_sampling=param_sampling,policy=early_termination_policy,primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=200)

In [7]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:
#TODO: Save the best model

import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Max iterations:',parameter_values[2])

joblib.dump(hyperdrive_run.get_hyperparameters(),'best_run_hd.pkl')

print(parameter_values)

Best Run Id:  HD_edb3b112-9f03-4834-ba88-ad5e5d880f88_142

 Accuracy: 0.925

 Max iterations: --max_iter
['--C', '0.7542410955939433', '--max_iter', '197']


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [5]:
from azureml.core.model import InferenceConfig, Model
from azureml.core import Environment
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.webservice import LocalWebservice

# Register the model to deploy
model_name ='hyperdrive_best_run'
description = 'AutoML model for predicting track recommendation'
tags={'area': "music", 'type': "classification"}
model = Model.register(model_name = model_name, 
                                  description = description, 
                                  tags = tags,model_path='best_run_hd.pkl',workspace=ws)

Registering model hyperdrive_best_run


In [6]:
# Managing directories and dependencies

source_directory = "source_directory"

os.makedirs(source_directory, exist_ok=True)
os.makedirs(os.path.join(source_directory, "dependencies/target"), exist_ok=True)
os.makedirs(os.path.join(source_directory, "env"), exist_ok=True)
os.makedirs(os.path.join(source_directory, "dockerstep"), exist_ok=True)

## Instantiate test data

Magic line method '%%writefile' must be first entry of cell

In [7]:
%%writefile source_directory/testdata.json

{
    "danceability":0.724,
    "energy":0.6,
    "key":1,
    "loudness":-6.25,
    "mode":0,
    "speechiness":0.087,
    "acousticness":0.28,
    "instrumentalness":6.83e-05,
    "liveness":0.108,
    "valence":0.201,
    "tempo":164.037
}

Writing source_directory/testdata.json


## Instantiate scoring script

Magic line method '%%writefile' must be first entry of cell

In [23]:
!pip install inference-schema

In [36]:
print(os.getenv('project_env'))

None


In [31]:
%%writefile ./source_directory/dependencies/target/score_hd.py
import joblib
import json
import numpy as np

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment. Join this path with the filename of the model file.
    # It holds the path to the directory that contains the deployed model (./azureml-models/$MODEL_NAME/$VERSION)
    # If there are multiple models, this value is the path to the directory containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('project_env'), 'best_run_hd.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)

    # Note here, the entire source directory from inference config gets added into image.
    # Below is an example of how you can use any extra files in image.
    with open('./source_directory/testdata.json') as json_file:
        data = json.load(json_file)

input_sample = {
    "danceability":0.724,
    "energy":0.6,
    "key":1,
    "loudness":-6.25,
    "mode":0,
    "speechiness":0.087,
    "acousticness":0.28,
    "instrumentalness":6.83e-05,
    "liveness":0.108,
    "valence":0.201,
    "tempo":164.037
}
output_sample = 0

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        prediction = 'Disliked'
        if result==0:
            prediction=prediction
        else:
            prediction='Liked'
        # You can return any JSON-serializable object.
        return "The track has features that you " + prediction
    except Exception as e:
        error = str(e)
        return error

Overwriting ./source_directory/dependencies/target/score_hd.py


# Deploy Model as a Docker Web service

NOTE:

The Docker image runs as a Linux container. If you are running Docker for Windows, you need to ensure the Linux Engine is running:

PowerShell command to switch to Linux engine
& `C:\Program Files\Docker\Docker\DockerCli.exe -SwitchLinuxEngine`

In [33]:
# Combine scoring script & environment in Inference configuration

hd_test_env = Environment(name="project-environment")

hd_test_env.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
hd_test_env.python.conda_dependencies.add_pip_package("joblib")
hd_test_env.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

# explicitly set base_image to None when setting base_dockerfile
hd_test_env.docker.base_image = None
hd_test_env.docker.base_dockerfile = "FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04\nRUN echo \"this is test\""
hd_test_env.inferencing_stack_version = "latest"

inference_config = InferenceConfig(entry_script="./source_directory/dependencies/target/score_hd.py",
                                   environment=hd_test_env)

# Set deployment configuration
deployment_config = LocalWebservice.deploy_configuration(port=6789)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(workspace = ws,
                       name = "my-web-service",
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = deployment_config)

service.wait_for_deployment()

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 0a4ff9bf382448e4b04e1e4e3e049b1a.azurecr.io
Logging into Docker registry 0a4ff9bf382448e4b04e1e4e3e049b1a.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 0a4ff9bf382448e4b04e1e4e3e049b1a.azurecr.io/azureml/azureml_3e92eb2a869925c317eb4b6ed4b5e2a4
 ---> e44a9549ecfc
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> c9ea2e498b8d
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImQ0YWQ3MjYxLTgzMmQtNDZiMi1iMDkzLTIyMTU2MDAxZGY1YiIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE1OTE4NCIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE1OTE4NCIsIndvcmtzcGFjZUlkIjoiMGE0ZmY5YmYtMzgyNC00OGU0LWIwNGUtMWU0ZTNlMDQ5YjFhIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 507d874b3eb5
 ---> c45a3548ab88
Step 4/5 : RUN mv '/var/azureml-app/tmpv9yqlhbz.py' /var/azureml-app/main.py


Error: Container has crashed. Did your init method fail?




Container Logs:
2021-09-24T14:22:22,482614350+00:00 - iot-server/run 
2021-09-24T14:22:22,483577662+00:00 - rsyslog/run 
2021-09-24T14:22:22,494447400+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-09-24T14:22:22,495114509+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/l

WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [17]:
print('Local service port: {}'.format(service.port))

Local service port: 6789


TODO: In the cell below, send a request to the web service you deployed to test it.

In [18]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
data = {
    
    "danceability":0.724,
    "energy":0.6,
    "key":1,
    "loudness":-6.25,
    "mode":0,
    "speechiness":0.087,
    "acousticness":0.28,
    "instrumentalness":6.83e-05,
    "liveness":0.108,
    "valence":0.201,
    "tempo":164.037
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)

print(response.json())

ConnectionError: HTTPConnectionPool(host='localhost', port=6789): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f7cc8e6cfd0>: Failed to establish a new connection: [Errno 111] Connection refused',))

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

service.delete()
model.delete()